In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,East London,ZA,2022-02-12 19:13:53,-33.0153,27.9116,77.43,86,0,12.86
1,1,Mangaratiba,BR,2022-02-12 19:13:10,-22.9597,-44.0406,81.03,85,68,4.29
2,2,Arraial Do Cabo,BR,2022-02-12 19:12:43,-22.9661,-42.0278,80.46,69,75,8.05
3,3,Tutoia,BR,2022-02-12 19:17:08,-2.7619,-42.2744,85.98,70,23,11.79
4,4,Castro,BR,2022-02-12 19:12:42,-24.7911,-50.0119,80.06,44,17,1.99


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [8]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,East London,ZA,2022-02-12 19:13:53,-33.0153,27.9116,77.43,86,0,12.86
1,1,Mangaratiba,BR,2022-02-12 19:13:10,-22.9597,-44.0406,81.03,85,68,4.29
2,2,Arraial Do Cabo,BR,2022-02-12 19:12:43,-22.9661,-42.0278,80.46,69,75,8.05
3,3,Tutoia,BR,2022-02-12 19:17:08,-2.7619,-42.2744,85.98,70,23,11.79
4,4,Castro,BR,2022-02-12 19:12:42,-24.7911,-50.0119,80.06,44,17,1.99
5,5,Carnarvon,AU,2022-02-12 19:11:06,-24.8667,113.6333,82.47,61,100,8.05
7,7,Hambantota,LK,2022-02-12 19:17:09,6.1241,81.1185,76.44,85,100,9.86
10,10,Rikitea,PF,2022-02-12 19:12:40,-23.1203,-134.9692,79.02,70,3,17.00
12,12,San Andres,CO,2022-02-12 19:17:10,12.5847,-81.7006,86.02,66,20,17.27
14,14,Salalah,OM,2022-02-12 19:17:11,17.0151,54.0924,75.29,64,75,6.91


In [14]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,East London,ZA,77.43,-33.0153,27.9116,
1,Mangaratiba,BR,81.03,-22.9597,-44.0406,
2,Arraial Do Cabo,BR,80.46,-22.9661,-42.0278,
3,Tutoia,BR,85.98,-2.7619,-42.2744,
4,Castro,BR,80.06,-24.7911,-50.0119,
5,Carnarvon,AU,82.47,-24.8667,113.6333,
7,Hambantota,LK,76.44,6.1241,81.1185,
10,Rikitea,PF,79.02,-23.1203,-134.9692,
12,San Andres,CO,86.02,12.5847,-81.7006,
14,Salalah,OM,75.29,17.0151,54.0924,


In [15]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [18]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))